In [ ]:
%matplotlib inline

# Create an MDO Scenario.


In [ ]:
from __future__ import annotations

from numpy import ones

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import create_scenario
from gemseo import get_available_opt_algorithms
from gemseo import get_available_post_processings

Let $(P)$ be a simple optimization problem:

$$
  (P) = \left\{
  \begin{aligned}
    & \underset{x}{\text{minimize}}
    & & f(x) = \sin(x) - \exp(x) \\
    & \text{subject to}
    & & -2 \leq x \leq 2
  \end{aligned}
  \right.
$$

In this subsection, we will see how to use GEMSEO to solve this problem
$(P)$ by means of an optimization algorithm.

## Define the discipline

Firstly, by means of the high-level function [create_discipline()][gemseo.create_discipline],
we create an [Discipline][gemseo.core.discipline.discipline.Discipline] of [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline] type
from a Python function:



In [ ]:
expressions = {"y": "sin(x)-exp(x)"}
discipline = create_discipline("AnalyticDiscipline", expressions=expressions)

We can quickly access the most relevant information of any discipline (name, inputs,
and outputs) with their string representations. Moreover, we can get the default
input values of a discipline with the attribute [Discipline.default_input_data][gemseo.core.discipline.discipline.Discipline.default_input_data:



In [ ]:
discipline, discipline.default_input_data

Now, we can minimize an output of this [Discipline][gemseo.core.discipline.discipline.Discipline] over a design space,
by means of a quasi-Newton method from the initial point $0.5$.

## Define the design space

For that, by means of the high-level function [create_design_space()][gemseo.create_design_space],
we define the [DesignSpace][gemseo.algos.design_space.DesignSpace] $[-2, 2]$ with initial value $0.5$
by using its [add_variable()][gemseo.algos.design_space.DesignSpace.add_variable] method.



In [ ]:
design_space = create_design_space()
design_space.add_variable("x", lower_bound=-2.0, upper_bound=2.0, value=-0.5 * ones(1))

## Define the MDO scenario

Then, by means of the [create_scenario()][gemseo.create_scenario] API function,
we define an [MDOScenario][gemseo.scenarios.mdo_scenario.MDOScenario] from the [Discipline][gemseo.core.discipline.discipline.Discipline]
and the [DesignSpace][gemseo.algos.design_space.DesignSpace] defined above:



In [ ]:
scenario = create_scenario(
    discipline, "y", design_space, formulation_name="DisciplinaryOpt"
)

## What about the differentiation method?

The [AnalyticDiscipline][gemseo.disciplines.analytic.AnalyticDiscipline] automatically differentiates the
expressions to obtain the Jacobian matrices. Therefore, there is no need to
define a differentiation method in this case. Keep in mind that for a
generic discipline with no defined Jacobian function, you can use the
[set_differentiation_method()][gemseo.scenarios.base_scenario.BaseScenario.set_differentiation_method] method
to define a numerical approximation of the gradients.

``` python
scenario.set_differentiation_method("finite_differences")
```

## Execute the MDO scenario

Lastly, we solve the [OptimizationProblem][gemseo.algos.optimization_problem.OptimizationProblem] included in the
[MDOScenario][gemseo.scenarios.mdo_scenario.MDOScenario] defined above by minimizing the objective function over
the [DesignSpace][gemseo.algos.design_space.DesignSpace].
Precisely, we choose the [L-BFGS-B algorithm](https://en.wikipedia.org/wiki/Limited-memory_BFGS) implemented in the
function `scipy.optimize.fmin_l_bfgs_b`.



In [ ]:
scenario.execute(algo_name="L-BFGS-B", max_iter=100)

The optimum results can be found in the execution log. It is also possible to
access them with [BaseScenario.optimization_result][gemseo.scenarios.base_scenario.BaseScenario.optimization_result]:



In [ ]:
optimization_result = scenario.optimization_result
f"The solution of P is (x*, f(x*)) = ({optimization_result.x_opt}, {optimization_result.f_opt})"

!!! info "See also"

    You can find the [SciPy](https://scipy.org/) implementation of the
    [L-BFGS-B algorithm](https://en.wikipedia.org/wiki/Limited-memory_BFGS)
    algorithm [by clicking here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.optimize.fmin_l_bfgs_b.html).

## Available algorithms

In order to get the list of available optimization algorithms, use:



In [ ]:
get_available_opt_algorithms()

## Available post-processing

In order to get the list of available post-processing algorithms, use:



In [ ]:
get_available_post_processings()

## Exporting the problem data.

After the execution of the scenario, you may want to export your data to use it
elsewhere. The [to_dataset()][gemseo.scenarios.base_scenario.BaseScenario.to_dataset] will allow you to export your
results to a [Dataset][gemseo.datasets.dataset.Dataset], the basic GEMSEO class to store data.



In [ ]:
dataset = scenario.to_dataset("a_name_for_my_dataset")

You can also look at the examples:

.. raw:: html

   <div style="text-align: center;"><a class="btn gemseo-btn mb-1"
   href="../post_process/index.html" role="button">Examples</a></div>

